## Squish Test
To perform the analysis, there are two stages of the processing:
- Segment bananas out from the remainder of the image
- Count the masked pixel area of this segmented section

We perform the first by running an edge detector algorithm followed by a watershed algorithm to extract out the complete mask of the object. Note that there are some nuances about how this edge detection is implemented: a vanilla Canny edge detector fails to work because of small issues with leaving "gaps" in the object boundary. 

The final "squish ratio" is defined as:

$$ \frac{\%pixels_{after}}{\%pixels_{before}} $$

In [1]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

In [2]:
def get_mask(hed, seed_point=(300, 250)):
    t = 0
    mask = hed.copy()
    mask[mask == t] = 0
    mask[mask > t] = 255

    mask[:, :200] = 0
    mask[:, 400:] = 0
    mask[:200, :] = 0
    mask[350:, :] = 0
    
    kernel = np.ones((5, 5), np.uint8)
    img_dilation = cv2.dilate(mask, kernel, iterations=10)
    mask = cv2.erode(img_dilation, kernel, iterations=10)

    cv2.floodFill(mask, None, seedPoint=seed_point, newVal=255)
    return mask

In [3]:
import urllib.request
import matplotlib.pyplot as plt

url = "https://github.com/ashukid/hed-edge-detector/raw/master/hed_pretrained_bsds.caffemodel"
urllib.request.urlretrieve(url, "hed_pretrained_bsds.caffemodel")

('hed_pretrained_bsds.caffemodel', <http.client.HTTPMessage at 0x7fa9f190d0a0>)

In [5]:
special_seeds = {
    "before": {
        2: (300, 300),
        3: (225, 275),
        4: (300, 300),
        5: (225, 275),
        7: (325, 275),
        8: (300, 300),
        9: (225, 275),
        10: (225, 275),
        11: (300, 300),
        15: (225, 225),
        20: (300, 300),
        29: (300, 300),
        36: (225, 275),
    },
    "after": {
        3: (300, 300),
        5: (225, 275),
        7: (350, 275),
        8: (275, 275),
        9: (225, 275),
        10: (225, 275),
        17: (325, 300),
        20: (300, 300),
        28: (300, 300),
        29: (300, 300),
    },
}

for folder in ["before", "after"]:
    for banana_idx in range(1, 37):
        root_folder = f"../SquishTest_Data"
        banana_fn = f"banana_{banana_idx}.jpeg"
        img_path = os.path.join(root_folder, folder, banana_fn)
        print(img_path)
        raw = cv2.imread(img_path)

        img = cv2.resize(raw, None, fx=0.15, fy=0.15)
        img = cv2.blur(img, (5,5)) 

        W, H, _ = img.shape
        blob = cv2.dnn.blobFromImage(img, scalefactor=1.0, size=(W, H), swapRB=False, crop=False)
        net = cv2.dnn.readNetFromCaffe("../deploy.prototxt", "hed_pretrained_bsds.caffemodel")

        net.setInput(blob)
        hed = net.forward()

        hed = cv2.resize(hed[0, 0], (W, H))
        hed = (255 * hed).astype("uint8")
        if banana_idx in special_seeds[folder]:
            seed_point = special_seeds[folder][banana_idx]
        else:
            seed_point = (300, 250)
        mask = get_mask(hed, seed_point=seed_point)

        dest = os.path.join(root_folder, "masked", folder, banana_fn)
        print(f"Writing to: {dest}")
        cv2.imwrite(dest, mask)

../SquishTest_Data/before/banana_1.jpeg
Writing to: ../SquishTest_Data/masked/before/banana_1.jpeg
../SquishTest_Data/before/banana_2.jpeg
Writing to: ../SquishTest_Data/masked/before/banana_2.jpeg
../SquishTest_Data/before/banana_3.jpeg
Writing to: ../SquishTest_Data/masked/before/banana_3.jpeg
../SquishTest_Data/before/banana_4.jpeg
Writing to: ../SquishTest_Data/masked/before/banana_4.jpeg
../SquishTest_Data/before/banana_5.jpeg
Writing to: ../SquishTest_Data/masked/before/banana_5.jpeg
../SquishTest_Data/before/banana_6.jpeg
Writing to: ../SquishTest_Data/masked/before/banana_6.jpeg
../SquishTest_Data/before/banana_7.jpeg
Writing to: ../SquishTest_Data/masked/before/banana_7.jpeg
../SquishTest_Data/before/banana_8.jpeg
Writing to: ../SquishTest_Data/masked/before/banana_8.jpeg
../SquishTest_Data/before/banana_9.jpeg
Writing to: ../SquishTest_Data/masked/before/banana_9.jpeg
../SquishTest_Data/before/banana_10.jpeg
Writing to: ../SquishTest_Data/masked/before/banana_10.jpeg
../Squis

In [6]:
squishes = []
for banana_idx in range(1, 37):
    root_folder = f"../SquishTest_Data"
    banana_fn = f"banana_{banana_idx}.jpeg"
    before_mask = cv2.imread(os.path.join(root_folder, "masked", "before", banana_fn), 0) > 0
    after_mask = cv2.imread(os.path.join(root_folder, "masked", "after", banana_fn), 0) > 0

    before_area = np.sum(before_mask)
    after_area = np.sum(after_mask)

    squish = after_area / before_area
    squishes.append(squish)
squishes = np.array(squishes)

index = [f"banana_{banana_idx}" for banana_idx in range(1, 37)]
df = pd.DataFrame(squishes, columns=["squish"], index=index)
df.to_csv("squish.csv")